# 我们来构造一个有memory的智能体

##  先构造一个GOOGLE搜索工具

In [2]:
from langchain_community.utilities import GoogleSearchAPIWrapper


search = GoogleSearchAPIWrapper()

# 注意tools是一个列表，只不过这里只有一个元素，回想一下上一节课的内容
tools = [
    Tool(
        name="Google Search",
        description="useful when you need to answer questions about current events",
        func=search.run,
    )
]
tools[0].run("what is the performance of NASDAQ today？")

"Join the Nasdaq+ community for more personalized data, exclusive content, and market analysis. Try it today for $4.99/month. Subscribe Now. 10,982.80 - 15,630.59. Performance. 5 Day. -2.23%. 1 Month. 4.51%. 3 Month. 14.06%. YTD. 1.02%. 1 Year. 24.29%. Recent News. MarketWatch. Dow Jones. Read full\xa0... Find the latest stock market trends and activity today. Compare key indexes, including Nasdaq Composite, Nasdaq-100, Dow Jones Industrial & more. Find the latest information on NASDAQ Composite (^IXIC) including data, charts, related news and more from Yahoo Finance. Results 1 - 25 of 7344 ... Join the Nasdaq+ community for more personalized data, exclusive content, and market analysis. Try it today for $4.99/month. Subscribe\xa0... ... Index. 4,845.65. - 79.32. 1.61%. NASDAQ Index. 15,164.01. - 345.89. 2.23%. Last updated Jan 31 at 5:27:42 PM ET. Today's hot stocks. Active Gainers Losers. Get the latest stock market news, stock information & quotes, data analysis reports, as well as 

## 构造一个ZeroShotAgent with memory

In [3]:
import os

from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain.agents import AgentExecutor,Tool,ZeroShotAgent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


prefix = """
    Have a conversation with a human, answering the following questions as best you can. 
    If you can answer the question by yourself,dont' use the tools.
    You have access to the following tools: """


suffix = """Begin!"

Previous Conversation:{chat_history}
Question: {input}
Thought: {agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)


memory = ConversationBufferMemory(memory_key="chat_history")
memory.clear()

memory.load_memory_variables({})

{'chat_history': ''}

In [4]:
llm = OpenAI(
    temperature=0,
    openai_api_key = os.getenv("OPENAI_API_KEY"),
    base_url = os.getenv("OPENAI_BASE_URL")
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory, max_iterations = 5
)

agent_chain.invoke({"input":"How many people live in China?"})

D:\chatbot\jupyter\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use Google Search to find the answer.
Action: Google Search
Action Input: "How many people live in China?"
Observation: China 2023 population is estimated at 1,425,671,352 people at mid year. ... The China Population (Live) counter shows a continuously updated estimate of ... Dec 5, 2022 ... Although China will lose its title as the world's most populous country, the UN still estimates its population at 1.426 billion people in 2022. Jul 10, 2017 ... 1. Good currency exchange rate and cheap cost of living well. · 2. Great food, the best in the world (in my opinion). · 3. Very friendly, warm, ... Jul 21, 2022 ... Global population projected to exceed 8 billion in 2022; half live in just seven countries ... People shown at a water park in ... Sep 2, 2020 ... "There is a feeling unavailable to people who were born in the United States: None of us can know what it's like to come from a country ... Apr 1, 2022 ... BEIJING, April 1, 2022— Over

{'input': 'How many people live in China?',
 'chat_history': '',
 'output': 'The estimated population of China in 2023 is 1,425,671,352 people.'}

In [5]:
agent_chain.invoke(input="What is the capital of that country?")



> Entering new AgentExecutor chain...
I should use Google Search to find the answer.
Action: Google Search
Action Input: "Capital of China"
Observation: Jun 4, 2015 ... Beijing is the capital of the People's Republic of China and one of the most populous cities in the world. The metropolis, located in north ... With about 22 million residents, Beijing is the world's most populous national capital city as well as China's second largest city after Shanghai. It is located ... Human Capital and China's Future Growth by Hongbin Li, Prashant Loyalka, Scott Rozelle and Binzhen Wu. Published in volume 31, issue 1, pages 25-48 of ... Beijing is the national capital, while Shanghai is the most populous city and largest financial center. People's Republic of China. 中华人民共和国 (Chinese). Aug 19, 2019 ... Capital misallocation has been documented as a prevailing empirical phenomenon both in less-developed economies in general and in China in ... Oct 18, 2023 ... WASHINGTON, DC – Chairman Mike Gallag

{'input': 'What is the capital of that country?',
 'chat_history': 'Human: How many people live in China?\nAI: The estimated population of China in 2023 is 1,425,671,352 people.',
 'output': 'The capital of China is Beijing.'}

In [7]:
agent_chain.invoke(input="who are you?")



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I should introduce myself.
Action: N/A`

## 尝试构造一个 create_react_agent

In [8]:
import os

from langchain_openai import OpenAI
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

template='''
Answer the following questions as best you can. 
Try to answer the question by yourself.
If not necessary, don't use the tools.

You have access to the following tools:{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous Conversation:{chat_history}
Question: {input}
Thought:{agent_scratchpad}
'''


prompt = PromptTemplate.from_template(template)
print(prompt)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'] template="\nAnswer the following questions as best you can. \nTry to answer the question by yourself.\nIf not necessary, don't use the tools.\n\nYou have access to the following tools:{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nPrevious Conversation:{chat_history}\nQuestion: {input}\nThought:{agent_scratchpad}\n"


In [9]:
llm = OpenAI(
    temperature=0,
    openai_api_key = os.getenv("OPENAI_API_KEY"),
    base_url = os.getenv("OPENAI_BASE_URL")
)

from langchain_community.utilities import GoogleSearchAPIWrapper


search = GoogleSearchAPIWrapper()


tools = [
    Tool(
        name="Google Search",
        description="A search engine you can use when you need to know latest news or data",
        func=search.run,
    ),
]

memory = ConversationBufferMemory(memory_key="chat_history")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory = memory, max_iterations = 3,handle_parsing_errors=True,verbose= True)

agent_executor.invoke({"input":"hi,how are you?"})




> Entering new AgentExecutor chain...
I should respond politely.
Action: NoneInvalid Format: Missing 'Action Input:' after 'Action:'I should respond politely.
Action: None
Action Input: NoneNone is not a valid tool, try one of [Google Search].I should respond politely.
Action: Google Search
Action Input: "how to respond politely"Dec 7, 2021 ... Instead of yes, you can say: · Yes I can/Yes, sure thing. · Yes of course!/Of course I will. · Yes I can. It's this way. · Sure. It's 10am. May 3, 2017 ... How to respond politely when someone tells you something useful? · thanks for letting me know · thanks for informing me · thanks for reminding ... Aug 14, 2019 ... Thank you for your advice, but I can handle this.” “I appreciate your help, but I prefer to do this on my own.” “I see what you're trying to ... Jun 15, 2022 ... The standard "response" would be ご無沙汰しております, which is "polite" and a little "formal" but this phrase/response what is usually/normally ... Jul 1, 2022 ... Possible appro

{'input': 'hi,how are you?',
 'chat_history': '',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [10]:
agent_executor.invoke({"input":"what is the population in USA?"})



> Entering new AgentExecutor chain...
Action: Google Search
Action Input: "population USA"Shows estimates of current USA Population overall and people by US state/county and of World Population overall, by country and most populated countries. The current population of the United States of America is 341,059,544 as of Wednesday, January 31, 2024, based on Worldometer elaboration of the latest United ... QuickFacts provides statistics for all states and counties. Also for cities and towns with a population of 5,000 or more. -- Select a fact -- ... With a population of over 334 million, it is the third-most populous country in the world. The national capital is Washington, D.C., and its most populous city ... Access demographic, economic and population data from the U.S. Census Bureau. ... USA Population (depicts US population). 335,996,532. World Population (depicts ... The Bureau of Population, Refugees, and Migration (PRM) is the humanitarian bureau of the U.S. Department of State. 

{'input': 'what is the population in USA?',
 'chat_history': 'Human: hi,how are you?\nAI: Agent stopped due to iteration limit or time limit.',
 'output': 'The current population of the United States of America is 341,059,544 as of Wednesday, January 31, 2024.'}